In [57]:
'''
import pyodbc
import pandas as pd
import numpy as np

#  Conexión a **Azure SQL**
AZURE_SERVER = 'uaxmathfis.database.windows.net'
AZURE_DATABASE = 'usecases'
AZURE_DRIVER = '{ODBC Driver 17 for SQL Server}'

azure_conn_str = f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};Authentication=ActiveDirectoryInteractive"

#  Conexión a **SQL Server LOCAL**
LOCAL_SERVER = 'localhost'
LOCAL_DATABASE = 'dwh_case1'  
LOCAL_DRIVER = '{ODBC Driver 17 for SQL Server}'

local_conn_str = f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes"

#  Consulta SQL en Azure SQL
SQL_QUERY = """
SELECT
[Id_Producto]
    ,producto.[Code_]
    ,producto.[CATEGORIA_ID]
    ,producto.[Modelo]
    ,fuel.[FUEL]
    ,categoría_producto.[Grade_ID]
    ,categoría_producto.[Equipamiento]
    ,costes.[Modelo]
    ,costes.[Costetransporte]
    ,costes.[GastosMarketing]
    ,costes.[Mantenimiento_medio]
    ,costes.[Comisión_Marca]

FROM [DATAEX].[006_producto] producto
LEFT JOIN
  [DATAEX].[015_fuel] fuel ON producto.Fuel_ID = fuel.Fuel_ID
LEFT JOIN
  [DATAEX].[014_categoría_producto] categoría_producto ON producto.CATEGORIA_ID = categoría_producto.CATEGORIA_ID
LEFT JOIN
  [DATAEX].[007_costes] costes ON producto.Modelo = costes.Modelo
"""

# 🔹 Nombre de la tabla en SQL Server Local
NEW_TABLE_NAME = "DATAEX.PRODUCT_DIM"

try:
    #  Conectar a Azure SQL
    print(f"Conectando a Azure SQL...")
    conn_azure = pyodbc.connect(azure_conn_str)
    
    # 🔹 Ejecutar la consulta en Azure SQL
    print(f"Ejecutando consulta en Azure SQL...")
    df = pd.read_sql(SQL_QUERY, conn_azure)

    if df.empty:
        print(f" La consulta no devolvió resultados. No se creará la tabla en SQL Server Local.")
    else:
        print(f"   - Datos extraídos: {df.shape[0]} filas")



        #  Convertir NaN en columnas numéricas a 0
        df = df.fillna(0)

        #  Convertir valores numéricos problemáticos
        for col in df.select_dtypes(include=['float64']).columns:
            df[col] = df[col].astype(np.float32)  # Reducir precisión
        
        for col in df.select_dtypes(include=['int64']).columns:
            df[col] = df[col].astype(np.int32)  # Evitar valores fuera de rango
        
        #  Conectar a SQL Server Local
        print(f"Conectando a SQL Server Local...")
        conn_local = pyodbc.connect(local_conn_str)
        
        with conn_local.cursor() as cursor:
            # 🔹 Eliminar la tabla si ya existe
            drop_table_sql = f"DROP TABLE IF EXISTS {NEW_TABLE_NAME}"
            cursor.execute(drop_table_sql)
            conn_local.commit()
            print(f"   - Tabla eliminada si existía.")

            # 🔹 Crear la tabla en SQL Server Local con tipos de datos ajustados
            create_table_sql = f"""
            CREATE TABLE {NEW_TABLE_NAME} (
                {', '.join([
                    f'[{col}] FLOAT' if df[col].dtype == np.float32 
                    else f'[{col}] INT' if df[col].dtype == np.int32 
                    else f'[{col}] NVARCHAR(255)' for col in df.columns
                ])}
            );
            """
            cursor.execute(create_table_sql)
            conn_local.commit()
            print(f" Tabla {NEW_TABLE_NAME} creada correctamente en SQL Server Local.")

            # Insertar los datos en SQL Server Local
            placeholders = ', '.join(['?' for _ in df.columns])
            insert_sql = f"INSERT INTO {NEW_TABLE_NAME} VALUES ({placeholders})"

            cursor.fast_executemany = True
            cursor.executemany(insert_sql, df.values.tolist())
            conn_local.commit()

            print(f" {df.shape[0]} filas insertadas en {NEW_TABLE_NAME}.")

except Exception as e:
    print(f" Error: {e}")

finally:
    if 'conn_azure' in locals():
        conn_azure.close()
    if 'conn_local' in locals():
        conn_local.close()

print("\n ¡Proceso completado!")
'''


'\nimport pyodbc\nimport pandas as pd\nimport numpy as np\n\n#  Conexión a **Azure SQL**\nAZURE_SERVER = \'uaxmathfis.database.windows.net\'\nAZURE_DATABASE = \'usecases\'\nAZURE_DRIVER = \'{ODBC Driver 17 for SQL Server}\'\n\nazure_conn_str = f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};Authentication=ActiveDirectoryInteractive"\n\n#  Conexión a **SQL Server LOCAL**\nLOCAL_SERVER = \'localhost\'\nLOCAL_DATABASE = \'dwh_case1\'  \nLOCAL_DRIVER = \'{ODBC Driver 17 for SQL Server}\'\n\nlocal_conn_str = f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes"\n\n#  Consulta SQL en Azure SQL\nSQL_QUERY = """\nSELECT\n[Id_Producto]\n    ,producto.[Code_]\n    ,producto.[CATEGORIA_ID]\n    ,producto.[Modelo]\n    ,fuel.[FUEL]\n    ,categoría_producto.[Grade_ID]\n    ,categoría_producto.[Equipamiento]\n    ,costes.[Modelo]\n    ,costes.[Costetransporte]\n    ,costes.[GastosMarketing]\n    ,costes.[Man

In [58]:
import pyodbc
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [59]:
# Azure connection string.
AZURE_SERVER = 'uaxmathfis.database.windows.net'
AZURE_DATABASE = 'usecases'
AZURE_DRIVER = '{ODBC Driver 17 for SQL Server}'

azure_conn_str = f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};Authentication=ActiveDirectoryInteractive"

In [60]:
# Local connection string.
LOCAL_SERVER = 'localhost'
LOCAL_DATABASE = 'dwh_case1'
LOCAL_DRIVER = '{ODBC Driver 17 for SQL Server}'

local_conn_str = f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes"

In [61]:
query_folder = "../Database/Dimensional"
queries = {
    "Dim_geo": "Dim_Geo.sql",
    "Dim_product": "Dim_Product.sql",
    "Dim_time": "Dim_Time.sql",
    "Dim_client": "Dim_Cli.sql",
    "Facts_Table": "Dim_Fact.sql"
}

In [62]:
# Primary keys for each table.
primary_keys = {
    "Facts_Table": ["CODE"],
    "Dim_client": ["Customer_ID"],
    "Dim_geo": ["TIENDA_ID"],
    "Dim_product": ["Id_Producto"],
    "Dim_time": ["Date"]
}

In [63]:
# Foreign keys for each table.
foreign_keys = {
    "Facts_Table": {
        "Customer_ID": "Dim_client(Customer_ID)",
        "TIENDA_ID": "Dim_geo(TIENDA_ID)",
        "Id_Producto": "Dim_product(Id_Producto)",
        "Sales_Date": "Dim_time(Date)"
    }
}

In [64]:
def create_table_sql(table_name, df):
    # Definición de los tipos de datos SQL para cada columna del DataFrame: Por defecto tipo TEXTO.
    col_defs = []
    for col in df.columns:
        if np.issubdtype(df[col].dtype, np.datetime64):
            col_defs.append(f'[{col}] DATE')
        elif df[col].dtype == np.float32:
            col_defs.append(f'[{col}] FLOAT')
        elif df[col].dtype == np.int32:
            col_defs.append(f'[{col}] INT')
        else:
            col_defs.append(f'[{col}] NVARCHAR(255)')

    # Agregación clave primaria si corresponde.
    pk = ", PRIMARY KEY (" + ", ".join(primary_keys[table_name]) + ")" if table_name in primary_keys else ""
    # Agregación claves foráneas si corresponde.
    fk = ""
    if table_name in foreign_keys:
        for col, ref in foreign_keys[table_name].items():
            fk += f", FOREIGN KEY ({col}) REFERENCES {ref}"

    return f"CREATE TABLE {table_name} ({', '.join(col_defs)}{pk}{fk});"

In [65]:
def drop_tables_in_order(cursor, conn):
    drop_order = ["Facts_Table", "Dim_time", "Dim_product", "Dim_geo", "Dim_client"]
    for table in drop_order:
        # Verifica si la tabla existe en el esquema actual.
        check_exists_query = f"""
        IF OBJECT_ID('{table}', 'U') IS NOT NULL
            DROP TABLE {table};
        """
        try:
            cursor.execute(check_exists_query)
            conn.commit()
        except Exception as e:
            print(f"Error al intentar eliminar la tabla {table}: {e}")

In [67]:
try:
    # Conexión a las bases de datos.
    conn_azure = pyodbc.connect(azure_conn_str)
    conn_local = pyodbc.connect(local_conn_str)
    print("Conexiones correctamente establecidas.\n")

    with conn_local.cursor() as cursor:
        drop_tables_in_order(cursor, conn_local)
    # Procesamiento de cada tabla definida en el diccionario de Queries.
    for table_name, file in queries.items():
        print(f"Procesando: {table_name}")
        query_path = os.path.join(query_folder, file)
        with open(query_path, "r", encoding="utf-8") as f:
            sql_query = f.read()

        # Ejecución de la consulta sobre la base de datos de Azure.
        df = pd.read_sql(sql_query, conn_azure)

        # Eliminación de las columnas duplicadas.
        if df.columns.duplicated().any():
            print(f"Columnas duplicadas en {table_name}: {df.columns[df.columns.duplicated()].tolist()}")
            df = df.loc[:, ~df.columns.duplicated()]

        # Detección de las columnas tipo DATE para convertirlas adecuadamente.
        for col in df.columns:
            if df[col].dtype == object or df[col].dtype == "string":
                sample_values = df[col].astype(str).sample(min(len(df), 30), random_state=42)
                # Saltar si parece una columna numérica (para no confundir INT con DATE).
                if sample_values.str.isdigit().mean() > 0.8:
                    continue
                try:
                    parsed = pd.to_datetime(sample_values, errors='coerce')
                    if parsed.notna().sum() > 0.9 * len(sample_values):
                        df[col] = pd.to_datetime(df[col], errors='coerce')
                except:
                    pass
        # Si el DataFrame está vacío, se salta.
        if df.empty:
            print(f"La tabla {table_name} no devolvió resultados.\n")
            continue
        print(f"   - Filas obtenidas: {df.shape[0]}")
        print(f"   - Columnas: {df.columns.tolist()}")

        # Limpieza de valores nulos y tipos de datos.
        for col in df.columns:
            df[col] = df[col].replace(r'^\s*$', np.nan, regex=True) # Reemplazar espacios en blanco por NaN.
            if pd.api.types.is_numeric_dtype(df[col]):
                # Valor sentinel (ej: -1 o 999999).
                sentinel = -1
                df[col] = df[col].fillna(sentinel)
            elif pd.api.types.is_datetime64_any_dtype(df[col]):
                df[col] = df[col].fillna(df[col].mode(dropna=True)[0])
            else:
                df[col] = df[col].fillna("N/A")
        for col in df.select_dtypes(include=['float64']).columns:
            df[col] = df[col].astype(np.float32)
        for col in df.select_dtypes(include=['int64']).columns:
            df[col] = df[col].astype(np.int32)

        # Creación de la tabla en la base de datos local.
        with conn_local.cursor() as cursor:
            create_sql = create_table_sql(table_name, df)
            cursor.execute(create_sql)
            conn_local.commit()
            print(f"   - Tabla {table_name} creada correctamente.")

            placeholders = ', '.join(['?' for _ in df.columns])
            insert_sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.fast_executemany = True
            cursor.executemany(insert_sql, df.values.tolist())
            conn_local.commit()
            print(f"   - {df.shape[0]} filas insertadas.\n")

except Exception as e:
    print(f"Error: {e}")

finally:
    if 'conn_azure' in locals():
        conn_azure.close()
    if 'conn_local' in locals():
        conn_local.close()

print("ETL completado.")

Conexiones correctamente establecidas.

Procesando: Dim_geo
   - Filas obtenidas: 12
   - Columnas: ['TIENDA_ID', 'PROVINCIA_ID', 'ZONA_ID', 'TIENDA_DESC', 'PROV_DESC', 'ZONA']
   - Tabla Dim_geo creada correctamente.
   - 12 filas insertadas.

Procesando: Dim_product
Columnas duplicadas en Dim_product: ['Modelo']
   - Filas obtenidas: 404
   - Columnas: ['Id_Producto', 'Code_', 'CATEGORIA_ID', 'Modelo', 'FUEL', 'Grade_ID', 'Equipamiento', 'Costetransporte', 'GastosMarketing', 'Mantenimiento_medio', 'Comisión_Marca']
   - Tabla Dim_product creada correctamente.
   - 404 filas insertadas.

Procesando: Dim_time
   - Filas obtenidas: 3652
   - Columnas: ['Date', 'Anno', 'Annomes', 'Dia', 'Diadelasemana', 'Diadelesemana_desc', 'Festivo', 'Findesemana', 'FinMes', 'InicioMes', 'Laboral', 'Mes', 'Mes_desc', 'Week']
   - Tabla Dim_time creada correctamente.
   - 3652 filas insertadas.

Procesando: Dim_client
   - Filas obtenidas: 44053
   - Columnas: ['Customer_ID', 'Edad', 'Fecha_nacimiento',